## Event based

In [3]:
from cartopy import config
import cartopy
import cartopy.crs as ccrs
import climtas
import dask.array
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from scipy import stats
from scipy.stats import ttest_ind 
import seaborn as sns
import sparse
import xarray as xr

In [4]:
# opening the input files with heatwave severity data 
thw = xr.open_dataset('/g/data/e14/cp3790/Charuni/Tasmania/aus-coastal-sev.nc')
mhw = xr.open_dataset('/g/data/e14/cp3790/Charuni/Tasmania/aus-ocean-sev-2.nc')

In [5]:
# using climtas.event.find_events to identify days where severity>1 for a minimum of 3 consecutive days 
thw_events = climtas.event.find_events(thw.severity > 1, min_duration = 3)
thw_events

,time,latitude,longitude,event_duration
0,0,48,151,3
1,0,48,152,3
2,0,50,152,3
3,0,54,154,3
4,0,54,155,3
...,...,...,...,...
165334,13508,130,142,6
165335,13508,130,143,6
165336,13508,131,142,6
165337,13508,133,140,6


In [6]:
# Create a sparse array with (coords, values, shape)
event_duration_array = sparse.COO([thw_events['time'], thw_events['latitude'], thw_events['longitude']], thw_events['event_duration'], shape=thw.severity.shape)
event_duration_da = xr.DataArray(event_duration_array, coords=thw.severity.coords)
event_duration_da

Format,coo
Data Type,int64
Shape,"(13514, 137, 165)"
nnz,165339
Density,0.0005412362553753639
Read-only,True
Size,5.0M
Storage ratio,0.0


In [7]:
## Need to work on this bit 

def get_coords_H(da, events):
    # Convert the index values to coordinates
    #da, events=event_duration_da, thw_events
    coords = {}
    for d in da.dims:
        coords[d] = da[d].values[events[d].values]

    # Also work out when the event1 	1982-08-08 	1982-08-10 	3 	1 	1982-08-08 	1982-08-10 	3 	 ends
    coords['time_end'] = da['time'].values[events['time'].values + events['event_duration'].values-1]
    coords['event_duration'] = events['event_duration'].values
    mean_severity=list()
    for i in range(len(events)):   
        end_time=coords['time_end'][i]
        du_time=coords['event_duration'][i]    
        delta = np.timedelta64(du_time-1,'D')    
        st_time=end_time-delta

        indices=list()
        for d in range(int(du_time)):
            index=np.where(np.asarray(da['time'])==st_time+np.timedelta64(d,'D'))[0][0]
            indices.append(index)
        mean_severity.append(da[indices].mean().values.ravel()[0])

    coords['severity'] = mean_severity
    coords
    return pd.DataFrame(coords, index=events.index)    

In [8]:
thw_new_H = get_coords_H(event_duration_da, thw_events)
thw_new_H

,time,latitude,longitude,time_end,event_duration,severity
0,1982-01-01,-22.00,150.75,1982-01-03,3,0.007240
1,1982-01-01,-22.00,151.00,1982-01-03,3,0.007240
2,1982-01-01,-22.50,151.00,1982-01-03,3,0.007240
3,1982-01-01,-23.50,151.50,1982-01-03,3,0.007240
4,1982-01-01,-23.50,151.75,1982-01-03,3,0.007240
...,...,...,...,...,...,...
165334,2018-12-26,-42.50,148.50,2018-12-31,6,0.003185
165335,2018-12-26,-42.50,148.75,2018-12-31,6,0.003185
165336,2018-12-26,-42.75,148.50,2018-12-31,6,0.003185
165337,2018-12-26,-43.25,148.00,2018-12-31,6,0.003185


In [9]:
thw_new_df = thw_new_H.reset_index()
thw_new_df.set_index(['latitude', 'longitude'], inplacesparse-Copy1=True)
thw_new_df

index       time   time_end  event_duration  severity
latitude longitude                                                        
-22.00   150.75          0 1982-01-01 1982-01-03               3  0.007240
         151.00          1 1982-01-01 1982-01-03               3  0.007240
-22.50   151.00          2 1982-01-01 1982-01-03               3  0.007240
-23.50   151.50          3 1982-01-01 1982-01-03               3  0.007240
         151.75          4 1982-01-01 1982-01-03               3  0.007240
...                    ...        ...        ...             ...       ...
-42.50   148.50     165334 2018-12-26 2018-12-31               6  0.003185
         148.75     165335 2018-12-26 2018-12-31               6  0.003185
-42.75   148.50     165336 2018-12-26 2018-12-31               6  0.003185
-43.25   148.00     165337 2018-12-26 2018-12-31               6  0.003185
         148.25     165338 2018-12-26 2018-12-31               6  0.003185

[165339 rows x 5 columns]